In [214]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import re
import numpy as np
from collections import Counter
import sys
import gc
from importlib import reload as rl
from functools import reduce
import spacy

sys.path.append('../')
import helper
import core
import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [146]:
rl(core.atos.base);
nlp = spacy.load('pt_core_news_sm')

In [147]:
csv_lis = []
for folder in os.listdir():
    if os.path.isdir(folder):
        for csv in os.listdir(folder):
            if (os.path.isfile(folder+'/'+csv)
                and csv.endswith('.csv')):
                csv_lis.append(folder+'/'+csv)

In [148]:
ctr = Counter([i.split('/')[0] for i in csv_lis])

df_dict = {}
for k in ctr:
    df_dict[k] = []

for csv in csv_lis:
    prf = csv.split('/')[0]
    df_dict[prf].append(pd.read_csv(csv))

In [149]:
df_text = {}
for k in df_dict:
    df_text[k] = []

for k, v in df_dict.items():
    for df in v:
        for t in df.texto:
            df_text[k].append(t)

df_filtered_text = {}
for k in df_dict:
    df_filtered_text[k] = []

for k, v in df_dict.items():
    for df in v:
        for t in df.texto:
            if len(t) < 1000:         
                df_filtered_text[k].append(t)


df_big_text = {}
for k in df_dict:
    df_big_text[k] = []

for k, v in df_dict.items():
    for df in v:
        for t in df.texto:
            if len(t) > 1000:         
                df_big_text[k].append(t)



In [150]:
whole = [(len(df_text[k]), k) for k in df_text]
lt = [(len(df_filtered_text[k]), k) for k in df_filtered_text]
gt = [(len(df_big_text[k]), k) for k in df_big_text]
print('WHOLE TEXT:')
for sz, key in sorted(whole, reverse=True):
    print(f'\t{sz} samples of {key}')

# print("LESS THAN 1000 text:")
# for sz, key in sorted(lt, reverse=True):
#     print(f'\t{sz} samples of {key}')
# print("GREATER THAN 1000 text:")
# for sz, key in sorted(gt, reverse=True):
#     print(f'\t{sz} samples of {key}')

del whole, lt, gt
gc.collect()

WHOLE TEXT:
	605 samples of Substituicao
	118 samples of Retirements
	61 samples of Exoneracao
	49 samples of NomeacaoComissionados
	25 samples of SemEfeitoAposentadoria
	10 samples of Cessoes
	9 samples of AbonoPermanencia
	2 samples of RetAposentadoria
	1 samples of Revertions


20

## A maior quantidade de casos extraídos é "subtituição". Porém, não foi possível realizar a curadoria das extrações. Por esta razão, serão utilizados os resultados de "semEfeitoAposentadoria"

In [151]:
sent_dict = {}
for k in df_text:
    sent_dict[k] = []
    for s in df_text[k]:
        sent_dict[k].extend(helper.sentencize_dodf(helper.preprocess(s)))
print("sentence number:")
for k, v in sorted(sent_dict.items(), key=lambda x:len(x[1]), reverse=True):
    print('\t',k, ':', len(v))

sentence number:
	 Substituicao : 605
	 Retirements : 304
	 Exoneracao : 61
	 Cessoes : 54
	 NomeacaoComissionados : 49
	 SemEfeitoAposentadoria : 40
	 AbonoPermanencia : 12
	 RetAposentadoria : 4
	 Revertions : 2


In [152]:
for ret in sent_dict['SemEfeitoAposentadoria']:
    print(ret, end=3*'\n')

TORNAR SEM EFEITO A RETIFICACAO DA AVERBACAO DE TEMPO DE SERVICO da servidora LUZIA SOUZA DOS PASSOS, matricula 01362518, AUXILIAR DE ENFERMAGEM, Secretaria de Estado de Saude, publicada no DODF No 243, quinta-feira, 21 de dezembro de 2017, pag. 42. .


TORNAR SEM EFEITO, na Ordem de Servico no 021, de 22 janeiro de 2020, publicada no DODF no 017, de 24/01/2020, pagina 24, o ato queRETIFICOUa AVERBACAO DE TEMPO DE SERVICO da servidora ANTONIA MARIA DA CONCEICAO, matricula no 135.843-X, AUXILIAR DE ENFERMAGEM, lotada no HMIB/SES, de 4.886 dias, ou seja, 13 anos, 4 meses e 21 dias, conforme certidao expedida pelo INSS, nos periodos de 01/09/1981 a 01/01/1984, de 15/03/1984 a 31/12/1984, de 14/01/1985 a 13/06/1987 e de 27/07/1989 a 31/05/1995, contados somente para fins de aposentadoria, de acordo com o processo no 272.000.401/2015. .


TORNAR SEM EFEITO, na Ordem de Servico no 021, de 22 janeiro de 2020, publicada no DODF no 017, de 24/01/2020, pagina 24, o ato queRETIFICOUa AVERBACAO DE

## Experimentação com xgboost (bons resultados previamente obtidos)

In [153]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import cross_validate

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import LabelEncoder

## IDEIA: supor todas as frases de um DODF n pertencentes da atos, ver resultados(podre, mas sem dados rotulados complica)

In [193]:
%%time
prefix = '../pdf/results/txt/'
all_matches = {'X': [], 'y':[]}
for txt in os.listdir(prefix):
    if txt.endswith('.txt'):
        content = open(prefix+'/'+txt).read()
        for k, v in core._dict.items():
            res = v('',txt=content)
            if res.data_frame.empty:
                continue
            else:
                all_matches['X'].extend(res.acts_str)
                all_matches['y'].extend(len(res.acts_str) * [k])

Counter(all_matches['y'])

CPU times: user 1.34 s, sys: 11.7 ms, total: 1.35 s
Wall time: 1.35 s


Counter({'substituicao': 605,
         'aposentadoria': 118,
         'nomeacao': 49,
         'exoneracao': 61,
         'abono': 9,
         'retificacoes': 2,
         'semEfeitoAposentadoria': 25,
         'cessao': 10,
         'reversoes': 1})

In [194]:
%%time
lis = os.listdir(prefix)
acc = 0
sents = {}
for pdf in lis:
    raw = open(prefix + pdf).read()
    _sents = helper.sentencize_dodf(raw)
    _sents = [i for i in _sents if i != i.upper()]
    _sents = [i for i in _sents if '....' not in i]
    sents[pdf] = _sents
    acc += len(_sents)
del _sents
gc.collect()
print("SENTENÇAS:", acc)

SENTENÇAS: 19009
CPU times: user 519 ms, sys: 0 ns, total: 519 ms
Wall time: 518 ms


## Tentar rotular sentenças com os extratores

In [197]:
%%time
sents_label = {'X':[], 'y':[]}

for k,v in sents.items():
    for s in v:
        for name, ext in core._dict.items():
            res = ext('', txt=s)
            if res.data_frame.empty:
                sents_label['X'].append(s)
                sents_label['y'].append("0")
            else:
                sents_label['X'].extend(res.acts_str)
                sents_label['y'].extend(len(res.acts_str)*[name])
Counter(sents_label['y'])

CPU times: user 47.8 s, sys: 2.09 ms, total: 47.8 s
Wall time: 47.8 s


Counter({'0': 170875,
         'exoneracao': 57,
         'semEfeitoAposentadoria': 24,
         'substituicao': 115,
         'aposentadoria': 10,
         'nomeacao': 2})

In [199]:
final = {
    'X': all_matches['X']+sents_label['X'],
    'y':all_matches['y']+sents_label['y']
}
df = pd.DataFrame(final)
df.y.value_counts()

0                         170875
substituicao                 720
aposentadoria                128
exoneracao                   118
nomeacao                      51
semEfeitoAposentadoria        49
cessao                        10
abono                          9
retificacoes                   2
reversoes                      1
Name: y, dtype: int64

In [202]:
final_df = df.copy()
final_df['y'] = final_df.y.apply(lambda x: 0 if x == '0' else 1)
final_df.y.value_counts()

0    170875
1      1088
Name: y, dtype: int64

In [203]:
%%time
X, y = final_df.X, final_df.y
pipe = Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('xgb',xgb.XGBClassifier(
        objective='multi:softprob',
#         num_class=len(df_text),        
        num_class=2,
    ))
])

CPU times: user 366 µs, sys: 8 µs, total: 374 µs
Wall time: 378 µs


In [206]:
%%time
cv = cross_validate(pipe, X, y, scoring=[
    'f1_macro', 'f1_micro', 'f1_weighted', 'accuracy']
)

CPU times: user 18min 17s, sys: 1.96 s, total: 18min 19s
Wall time: 3min 8s


In [209]:
fit_time = cv['fit_time']
del cv['fit_time']

In [215]:
for k,v in cv.items():
    if k.startswith('test'):
        print(k)
        print('\t', *[round(i, 2) for i in v])


test_f1_macro
	 0.82 0.96 0.8 0.91 0.95
test_f1_micro
	 0.99 1.0 0.99 1.0 1.0
test_f1_weighted
	 0.99 1.0 0.99 1.0 1.0
test_accuracy
	 0.99 1.0 0.99 1.0 1.0


## Montar base de sentenças rotuladas

In [155]:
rl(helper)
rl(core.atos)
rl(core)
rl(core.atos.base)
found = []
SemEfeitoAposentadoria = core._dict['semEfeitoAposentadoria']

In [159]:
len(core._dict['semEfeitoAposentadoria']('', 
    txt=open('../pdf/results/txt/02-04-2019_canvas.txt').read()
).acts_str)

5

In [160]:
esub = core._dict['substituicao']

all_subs = esub('', 
    txt=open('../pdf/results/txt/02-04-2019_canvas.txt').read()
)

t = open('../pdf/results/txt/02-04-2019_canvas.txt').read()

lis = []
for s in helper.sentencize_dodf(t):
    res = esub('', txt=s)
    if not res.data_frame.empty:
        lis.extend(res.acts_str)

In [161]:
print(all_subs._file_name)
for act in all_subs.acts_str:
    print('\t', act, end='\n\n')


	 Designar SIDELE DE JESUS SILVEIRA, Ouvidora, matricula 1.691.643-3, ANTONIO CELIO
RODRIGUES PIMENTEL, Chefe de Gabinete, matricula 1.689.965-7 e BARBARA GOMIDE
ANDRADE, Chefe da Assessoria de Comunicacao, matricula 1.689.566-5, para sob a presidencia do
primeiro comporem Grupo de Trabalho responsavel pela elaboracao e monitoramento da Carta de Servicos
a Administracao Regional de Planaltina RA-VI, conforme Decreto no 36.419/2015 e reiterado pela Lei
Federal no 13.460/2017

	 Designar VILMAR AMANCIO DE OLIVEIRA, matricula n 175.676-1, para atuar como Executor
e MARCELO ALVES DE SOUSA matricula n 1.690.189-4, como Executor Suplente, do contrato de
prestacao de servicos de fornecimento de energia eletrica entre esta Administracao Regional do Guara e a
CEB Distribuicao S/A (processo n137-00000630/2019-57)

	 Designar RAIMUNDO FRANCISCO ALVES ABREU, matricula no 1.689.978-4, Gerente da
Gerencia de Cultura, Esporte e Lazer, e RENATO ALEXANDRE CORTES JORGE, como executor e
suplente, respec

In [162]:
for i in lis:
    print('\t', i, end='\n\n\n')

	 Designar os servidores relacionados abaixo, com a indicacao das respectivas localidades, para
atuarem como Executores do Contrato no 05/2018, celebrado entre o Distrito Federal, e a empresa
CENTRO DE INTEGRACAO EMPRESA ESCOLA (CIEE), cujo objeto e a contratacao de agente de
integracao para operacionalizacao do Programa de Estagio da Secretaria de Estado de Fazenda,
Planejamento, Orcamento e Gestao do Distrito Federal mediante concessao de Bolsa-Auxilio, a alunos
regularmente matriculados e com frequencia efetiva em cursos de ensino regular, oferecidos por
instituicoes de educacao superior, de educacao profissional, de ensino medio, da educacao especial,
vinculados a estrutura do ensino publico e particular, Processo SEI no 00410-00015470/2017-10, a
saber:
1.RAIMUNDA IRIENE RODRIGUES TEIXEIRA BRAGA, Gestor em Politicas Publicas e Gestao
Governamental, matricula no 137.164-9 e ROSANGELA GOMES SANTANA SOUSA, Gestor em
Politicas Publicas e Gestao Governamental, matricula no 175.284-7, pa

In [163]:
len(lis)

7

In [164]:
!pwd

/home/leomaffei/Desktop/projeto-ia-unb/experiments/members/maffei/acts-classification/csv_better


In [167]:
df_sea = pd.DataFrame()

# not_sea = {'X': sents, 'y': len(sents) * [0]}
not_sea = {'X': [], 'y': []}

for s in sents:
    not_sea['X'].append(s)
    if 'PROCESSO' in s or re.search(r'\bresolve\b', s):
        not_sea['y'].append(1)
    else:
        not_sea['y'].append(0)
        

for k in df_text:
    for t in sent_dict[k]:
        not_sea['X'].append(t)
        not_sea['y'].append(1)

df_sea = pd.DataFrame(not_sea)
df_sea.head(), df_sea.y.value_counts()

(                                                   X  y
 0                              02-04-2019_canvas.txt  0
 1                              2020_04_30_canvas.txt  0
 2  NOMEAR ANA MARIA DE LIMA RODRIGUES, Auditor-Fi...  1
 3  NOMEAR ANDRE WILLIAM NARDES MENDES, Auditor-Fi...  1
 4  NOMEAR GERIVALDO ALVES MAGALHAES, Auditor-Fisc...  1,
 1    1131
 0       2
 Name: y, dtype: int64)

In [168]:
# label transformer (One vs ALL)
# df_sea['y'] = df_sea.y.apply(
#     lambda x: 1 if x == 'SemEfeitoAposentadoria' else 0)

# df_sea['y'] = LabelEncoder().fit_transform(df_sea.y)
# df_sea.y = 1
df_sea.y.value_counts()

1    1131
0       2
Name: y, dtype: int64

In [ ]:
processos = [i for i in sents if 'PROCESSO' in i]
processos